In [2]:
import itertools

import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric, MDSSClassificationMetric
from aif360.detectors import bias_scan

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

In [3]:
dataset_orig = load_preproc_data_compas()

female_group = [{'sex': 1}]
male_group = [{'sex': 0}]

In [4]:
dataset_orig_df = pd.DataFrame(dataset_orig.features, columns=dataset_orig.feature_names)

age_cat = np.argmax(dataset_orig_df[['age_cat=Less than 25', 'age_cat=25 to 45',
                                     'age_cat=Greater than 45']].values, axis=1).reshape(-1, 1)
priors_count = np.argmax(dataset_orig_df[['priors_count=0', 'priors_count=1 to 3',
                                          'priors_count=More than 3']].values, axis=1).reshape(-1, 1)
c_charge_degree = np.argmax(dataset_orig_df[['c_charge_degree=M', 'c_charge_degree=F']].values, axis=1).reshape(-1, 1)

features = np.concatenate((dataset_orig_df[['sex', 'race']].values, age_cat, priors_count,
                           c_charge_degree, dataset_orig.labels), axis=1)
feature_names = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']

In [5]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,two_year_recid
0,0.0,0.0,1.0,0.0,1.0,1.0
1,0.0,0.0,0.0,2.0,1.0,1.0
2,0.0,1.0,1.0,2.0,1.0,1.0
3,1.0,1.0,1.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,1.0,0.0


In [6]:
from aif360.datasets import StandardDataset
dataset = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

In [7]:
dataset_orig_train, dataset_orig_test = dataset.split([0.7], shuffle=True, seed=0)

In [8]:
metric_train = BinaryLabelDatasetMetric(dataset_orig_train,
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train.mean_difference())
metric_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=male_group,
                             privileged_groups=female_group)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.124496
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.159410


In [9]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

LogisticRegression(random_state=0)

In [10]:
clf.classes_

array([0., 1.])

In [11]:
dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

In [12]:
df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)
df.head()

,sex,race,age_cat,priors_count,c_charge_degree,observed,probabilities
0,1.0,1.0,2.0,2.0,1.0,1.0,0.552951
1,1.0,0.0,1.0,0.0,1.0,0.0,0.740959
2,0.0,1.0,0.0,1.0,1.0,0.0,0.374728
3,0.0,0.0,2.0,2.0,1.0,1.0,0.444487
4,0.0,1.0,1.0,1.0,0.0,1.0,0.584908


In [13]:
dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels